> INVESTING.COM 에서  NASDAQ, S&P, CBOE, 환율, 2년 만기 미국채 선물, 10년 만기 미국채 선물 가져오기

# API

In [1]:
import FinanceDataReader as fdr
from datetime import date, timedelta

In [2]:
today = date.today().strftime('%Y%m%d')
yesterday = (date.today()-timedelta(1)).strftime('%Y%m%d')

In [3]:
nasdaq = fdr.DataReader('NASDAQCOM', start='2021-09-28', data_source='fred')
nasdaq

,NASDAQCOM
DATE,
2021-09-28,14546.68
2021-09-29,14512.44
2021-09-30,14448.58


In [4]:
sp500 = fdr.DataReader('US500', yesterday, today)
sp500

,Close,Open,High,Low,Volume,Change
Date,,,,,,
2021-10-01,4357.05,4317.16,4375.19,4288.52,0.0,0.0115


In [5]:
cboe = fdr.DataReader('VIX', yesterday, today) # S&P 500 VIX
cboe

,Close,Open,High,Low,Volume,Change
Date,,,,,,
2021-10-01,21.15,23.14,24.89,20.64,0.0,-0.086


In [6]:
rate = fdr.DataReader('USD/KRW', yesterday, today)
rate

,Close,Open,High,Low,Change
Date,,,,,
2021-10-01,1180.35,1184.78,1189.05,1179.2,-0.0033


In [7]:
tmp = rate.reset_index()
tmp

,Date,Close,Open,High,Low,Change
0,2021-10-01,1180.35,1184.78,1189.05,1179.2,-0.0033


## 함수화

In [8]:
import pandas as pd
import sqlite3
import FinanceDataReader as fdr
from datetime import date, timedelta


class INVESTING:
    def getDate(self):
        self.end = date.today().strftime('%Y%m%d')  # 오늘
        self.start = (date.today() - timedelta(1)).strftime('%Y%m%d')  # 어제

    def getStock(self):
        self.sp500 = fdr.DataReader('US500', yesterday, today)
        self.sp500 = self.sp500[['Close']]

        self.cboe = fdr.DataReader('VIX', yesterday, today)
        self.cboe = self.cboe[['Close']]

        self.rate = fdr.DataReader('USD/KRW', yesterday, today)
        self.rate  = self.rate[['Close']]

    def saving(self):
        self.df = pd.concat([self.sp500, self.cboe, self.rate], axis=1)
        self.df_invest = self.df.reset_index()
        self.df_invest.columns=['date', 'sp', 'cboe', 'exchangerate']
        self.df_invest = self.df_invest.set_index('date')
        print(self.df_invest)
#         self.df.to_sql('samsung', conn, if_exists='append')
#         conn.commit()  # db에 저장


if __name__ == "__main__":
    
    invest = INVESTING()
    invest.getDate()
    invest.getStock()
    invest.saving()

                 sp   cboe  exchangerate
date                                    
2021-10-01  4357.05  21.15       1180.35


# Scraping

In [9]:
# !pip install selenium

from selenium import webdriver
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time

### 2년 만기 미국채 선물

In [10]:
content = []

In [11]:
# webdirver 설정

driver = webdriver.Chrome('C:\chromedriver.exe')
driver.get('https://kr.investing.com/rates-bonds/us-2-yr-t-note-historical-data')
driver.set_window_size(1080,1080)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

us2yr = soup.select('#curr_table > tbody > tr:nth-child(1) > td:nth-child(2)')
us2yr[0].text.strip()
content.append(us2yr[0].text.strip())

del soup
driver.close()

### 10년 만기 미국채 선물

In [12]:
driver = webdriver.Chrome('C:\chromedriver.exe')
driver.get('https://kr.investing.com/rates-bonds/us-10-yr-t-note-historical-data')
driver.set_window_size(1080,1080)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

us10yr = soup.select('#curr_table > tbody > tr:nth-child(1) > td:nth-child(2)')
us10yr[0].text.strip()
content.append(us10yr[0].text.strip())

del soup
driver.close()

In [13]:
content

['110.08', '132.20']

### 날짜 인덱스 부여

In [14]:
from datetime import date, datetime, timedelta

#date = datetime.today().replace(hour=0, minute=0, second=0)
#date = pd.DatetimeIndex([date.strftime('%Y-%m-%d %H:%M:%S')])
date = pd.DatetimeIndex([datetime.today().date()])
date

DatetimeIndex(['2021-10-02'], dtype='datetime64[ns]', freq=None)

In [15]:
df = pd.DataFrame([content], date, ['futures2y','futures10y'])
df = df.reset_index().rename(columns={'index': 'date'})
df = df.set_index('date')
df

,futures2y,futures10y
date,,
2021-10-02,110.08,132.20


## 함수화

In [16]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
from datetime import date, datetime, timedelta


class INDEXCRAWL:
    def __init__(self):
        self.contents = []
    
    def crawling(self,url):
        self.driver = webdriver.Chrome('C:\chromedriver.exe')
        self.driver.get(url)
        self.driver.set_window_size(720,720) 
        #self.driver.execute_script("window.scrollTo(0, 700)")
        
        self.soup = BeautifulSoup(self.driver.page_source, 'html.parser')
        self.index = self.soup.select('#last_last')
        #self.index = self.soup.select('#curr_table > tbody > tr:nth-child(1) > td:nth-child(2)')
        self.contents.append(self.index[0].text.strip())
        
        del self.soup
        self.driver.quit()
        
        
    def saving(self):
        self.date = pd.DatetimeIndex([datetime.today().date()])
        self.df = pd.DataFrame([self.contents], self.date, ['futures2y','futures10y'])
        print(self.df)
        
    

if __name__ == "__main__":
    crawl = INDEXCRAWL()
    us2yr ='https://kr.investing.com/rates-bonds/us-2-yr-t-note-historical-data'
    crawl.crawling(us2yr)
    us10yr = 'https://kr.investing.com/rates-bonds/us-10-yr-t-note-historical-data'
    crawl.crawling(us10yr)
    crawl.saving()

           futures2y futures10y
2021-10-02    110.08     132.20
